# Power Quality Classification using Muti Layer Perceptron (Dataset 1)

This notebook focusses on developing a Multi Layer perceptron which classifies a particular power signal into its respective power quality condition. The dataset used here contains signals which belong to one of the 5 classes(power quality condition). The sampling rate of this data is 128. This means that each signal is characterized by 128 data points. Here the signals provided are in time domain.

The power quality condition with respect to the output class value is as follows: <br>
1 - Normal<br>
2 - 3rd harmonic wave<br>
3 - 5th harmonic wave<br>
4 - Voltage dip<br>
5 - transient<br>

In [ ]:
#importing the required libraries
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
import datetime
from scipy.fft import fft,fftfreq
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.optimizers import Adam

In [ ]:
#loading the dataset using pandas
data = pd.read_csv("../input/powerqualitydistributiondataset1/PowerQualityDistributionDataset1.csv")

In [ ]:
#The dataset is already preprocessed
data.drop(data.columns[[0]],axis=1,inplace=True)
data.shape

In [ ]:
data.head()

In [ ]:
#here we are constructing the array which will finally contain the column names
header =[]
for i in range(1,129):
    header.append("Col"+str(i))
data_out = data['output']    

In [ ]:
data.drop(['output'],axis=1,inplace=True)
data_arr = data.to_numpy()

In [ ]:
data_arr.shape

### Data transformation

The data transformation steps employed here are as follows:<br>

1) Fourier Transform<br>
2) Normalization


In [ ]:
#In this segment we are plotting one wave from each class after applying fourier transformation 
w1 = data_arr[0][0:128]
w1[0:128] = np.abs(fft(w1[0:128]))
xf = fftfreq(128,1/128)
plt.plot(xf, w1)
plt.show()
print("class",data_out[0], "Normal wave")

w2 = data_arr[1][0:128]
w2[0:128] = np.abs(fft(w2[0:128]))
xf = fftfreq(128,1/128)
plt.plot(xf, w2)
plt.show()
print("class",data_out[1], "3rd harmonic wave")

w3 = data_arr[3][0:128]
w3[0:128] = np.abs(fft(w3[0:128]))
xf = fftfreq(128,1/128)
plt.plot(xf, w3)
plt.show()
print("class",data_out[3], "5th harmonic wave")

w4 = data_arr[6][0:128]
w4[0:128] = np.abs(fft(w4[0:128]))
xf = fftfreq(128,1/128)
plt.plot(xf, w4)
plt.show()
print("class",data_out[6], "Voltage dip")

w5 = data_arr[8][0:128]
w5[0:128] = np.abs(fft(w5[0:128]))
xf = fftfreq(128,1/128)
plt.plot(xf, w5)
plt.show()
print("class",data_out[8], "Transient wave")

In [ ]:
#here we are overwritting the dataframe with the waves which we obtained after doing fourier transformation
n = data_arr.shape[0]
for i in range(0,n):
    data_arr[i][0:128] = np.abs(fft(data_arr[i][0:128]))

In [ ]:
data_arr.shape

In [ ]:
#here we are performing normalization
transform = StandardScaler()
data_arr = transform.fit_transform(data_arr)

In [ ]:
#converting the numpy array back to data frame
data = pd.DataFrame(data_arr,columns=header)
data['output'] = data_out

In [ ]:
data

## Model creation and training

In [ ]:
#here we are splitting the dataset in the ratio of 60%,20%,20% (training set,validation set, test set)
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data.loc[:,data.columns != 'output'],data['output'],test_size=0.2)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25, random_state=42)


In [ ]:
# get_dummies function is used here to perform one hot encoding of the y_* numpy arrays
y_train_hot = pd.get_dummies(y_train)
y_test_hot = pd.get_dummies(y_test)
y_val_hot = pd.get_dummies(y_val)

In [ ]:
print("Training",x_train.shape)
print(y_train_hot.shape)
print("Validation",x_val.shape)
print(y_val_hot.shape)
print("Test",x_test.shape)
print(y_test_hot.shape)

In [ ]:
model = Sequential()
model.add(Dense(64, input_shape=(128,), activation = 'relu'))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(16, activation = 'relu'))
model.add(Dense(5, activation = 'softmax'))


In [ ]:
log_dir = "logs1/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

In [ ]:
model.summary()

In [ ]:
history = model.fit(x_train, y_train_hot, batch_size=64, epochs=30, validation_data=(x_val, y_val_hot), callbacks=[tensorboard_callback])

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs1/fit

In [ ]:
print(model.metrics_names)

## Model evaluation

In [ ]:
np.mean(history.history['val_accuracy']) 

In [ ]:
pred_acc = model.evaluate(x_test,y_test_hot)
print("Test accuracy is {}".format(pred_acc))

In [ ]:
x_test_arr = x_test.to_numpy()
predict = model.predict(x_test_arr[0:10][:])
predict_class = np.argmax(predict, axis=1)
predict_class = np.array(predict_class.tolist())
print(predict_class+1)